In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler#Keras
import keras
import sklearn
import librosa
import pandas as pd
import numpy as np
import scipy as sp
from keras import models
from keras import layers
import os
import random


In [3]:
#import librosa.feature
def extract_features(filename):
    """ function to extract the features from the wav files stored in ./all_samples/
        
        Args:
            filename (string): the name of the csv file in which the data samples with their new features
            will be stored
            
        Returns:
            None
    
    """
    
    
    header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
    for i in range(1, 21):
        header += f' mfcc{i}'
    header += ' label'
    header = header.split()
    
    RX = []
    Deformed = []

    file = "./all_samples/sample_A_"
    for i in range(1,4):
        for j in range(57):
            RX.append(file + "RX" + str(i) + "_C0" + str(j) + ".wav")
            Deformed.append(file + "DEFORMED" + str(i) + "_C0" + str(j) + ".wav")

    file = open(filename, 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
    for g in [(RX,"RX"),(Deformed,"Deformed")]:
        label = g[1]
        g = g[0]
        for filename in g:
            y, sr = librosa.load(filename, mono=True)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            to_append += f' {label}'
            file = open(filename, 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

In [6]:
def split_dataset(filename, train_set_percentage_of_data):
    """ function to split the given dataset.Note that the it assigns the label 1 to the class RX and the label 0 
        to the classe deformed. 
        
        Args:
            filename (string): the name of the csv file in which the data samples are stored. This data must be in
            the same format as the export of the extract features function for the slit to work. 
            
            train_set_percentage_of_data (float): the percentage of the data which will be assigned to the training
            set.
            
        Returns:
            (X_train, X_test, y_train, y_test): np arrays of the data samples and their labels splited into a 
            training set and a test set. 
    
    """
    
    data = pd.read_csv(filename)
    data = data.drop(['filename'],axis=1)
    labels = data.iloc[:, -1]
    encoder = LabelEncoder()
    y = encoder.fit_transform(labels)
    #print(y)
    #print(data['label'])

    scaler = StandardScaler()
    X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

    X_train, X_test, y_train, y_test = [],[],[],[]

    train_set_proportion = int(train_set_percentage_of_data * 57) #out of 57
    print(train_set_proportion)
    lower_bound = random.choice(list(np.arange(57)))

    upper_bound = lower_bound + train_set_proportion




    swapped = False
    if upper_bound >= 57:
        upper_bound = upper_bound - 57
        swapped = True



    print("(" + str(lower_bound) + ", " + str(upper_bound) + ")" )
    indices = []
    for i in range(len(X)):
        if ((not swapped) and (i % 57 < upper_bound and i % 57 > lower_bound)) or (swapped and (i % 57 < upper_bound or i % 57 > lower_bound)) :
            X_train.append(X[i])
            y_train.append(y[i])
            indices.append(i + 2)
        else:
            X_test.append(X[i])
            y_test.append(y[i])
            
    return np.asarray(X_train), np.asarray(X_test), np.asarray(y_train), np.asarray(y_test)

In [8]:
def build_model(optimizer, loss):
    """ function to build a NN model
        
        Args:
            optimizer (string): the name of the optimizer to be used for the model
            
            loss (string): the name of the loss function to be used for the model
            
        Returns:
            model: the model built with the given paramters
            
    """
    
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))


    model.compile(optimizer,loss,metrics=['accuracy'])
    return model

In [20]:
"""The model trained with the optimizer we decided to be the best on a random split"""

X_train,X_test, y_train, y_test = split_dataset("data.csv",0.7)

model = build_model('rmsprop', 'sparse_categorical_crossentropy')

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

39
(34, 16)
Epoch 1/20
2/2 [==============================] - 0s 2ms/step - loss: 0.6202 - accuracy: 0.7281
Epoch 2/20
2/2 [==============================] - 0s 3ms/step - loss: 0.3550 - accuracy: 0.9737
Epoch 3/20
2/2 [==============================] - 0s 4ms/step - loss: 0.2671 - accuracy: 0.9781
Epoch 4/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1581 - accuracy: 0.9868
Epoch 5/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1035 - accuracy: 0.9912
Epoch 6/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0839 - accuracy: 0.9868
Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0688 - accuracy: 0.9825
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0653 - accuracy: 0.9868
Epoch 9/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0478 - accuracy: 0.9956
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0339 - accuracy: 0.9912
Epoch 11/20
2/2 [==========

In [21]:
"""The model evaluated on the random test set"""
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

4/4 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 1.0000
test_acc:  1.0


In [22]:
def test_multiple_splits(optimizer, loss, iterations, training_set_percentage):
    """ function to test a model for given optimizer
        
        Args:
            optimizer (string): the name of the optimizer to be used for the model
            
            loss (string): the name of the loss function to be used for the model
            
            iterations (Integer): the number of random splits on which the model is tested
            
            train_set_percentage_of_data (float): the percentage of the data which will be 
            assigned to the training set.
            
        Returns:
            (accuracies,losses): the lists of accuracies and losses for every split
            
    """
    accuracies = np.empty(0)
    losses = np.empty(0)

    for i in range(iterations):
    
        model = build_model(optimizer,loss)

        X_train,X_test, y_train, y_test = split_dataset("data.csv",0.7)

        model.fit(X_train, y_train, epochs=20, batch_size=128)
        
        test_loss, test_acc = model.evaluate(X_test,y_test)
    
        accuracies = np.append(accuracies,test_acc)
        losses = np.append(losses,test_loss)
    
    return accuracies,losses

In [30]:
"""Test all optimizers for 100 iterations each with 70% of the data for the training set and sparse categorical
   crossentropy as loss function 
"""

optimizers = ["adam","sgd", "rmsprop"]
results = {}

for opt in optimizers:
    accuracies,losses = test_multiple_splits(opt, 'sparse_categorical_crossentropy', 100, 0.7)
    results[opt] = (accuracies,losses)

39
(43, 25)
Epoch 1/20
2/2 [==============================] - 0s 2ms/step - loss: 0.7136 - accuracy: 0.5833
Epoch 2/20
2/2 [==============================] - 0s 3ms/step - loss: 0.5554 - accuracy: 0.9737
Epoch 3/20
2/2 [==============================] - 0s 4ms/step - loss: 0.4507 - accuracy: 0.9693
Epoch 4/20
2/2 [==============================] - 0s 4ms/step - loss: 0.3553 - accuracy: 0.9737
Epoch 5/20
2/2 [==============================] - 0s 4ms/step - loss: 0.2734 - accuracy: 0.9825
Epoch 6/20
2/2 [==============================] - 0s 4ms/step - loss: 0.2026 - accuracy: 0.9868
Epoch 7/20
2/2 [==============================] - 0s 5ms/step - loss: 0.1459 - accuracy: 0.9912
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1028 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0717 - accuracy: 0.9956
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0490 - accuracy: 0.9956
Epoch 11/20
2/2 [==========

2/2 [==============================] - 0s 2ms/step - loss: 0.6856 - accuracy: 0.5921
Epoch 2/20
2/2 [==============================] - 0s 2ms/step - loss: 0.4973 - accuracy: 0.9956
Epoch 3/20
2/2 [==============================] - 0s 3ms/step - loss: 0.3595 - accuracy: 0.9956
Epoch 4/20
2/2 [==============================] - 0s 2ms/step - loss: 0.2554 - accuracy: 0.9868
Epoch 5/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1745 - accuracy: 0.9912
Epoch 6/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1148 - accuracy: 0.9912
Epoch 7/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0732 - accuracy: 0.9956
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0462 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0299 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0187 - accuracy: 1.0000
Epoch 11/20
2/2 [==============================] -

2/2 [==============================] - 0s 2ms/step - loss: 0.6464 - accuracy: 0.6535
Epoch 2/20
2/2 [==============================] - 0s 3ms/step - loss: 0.5016 - accuracy: 0.9430
Epoch 3/20
2/2 [==============================] - 0s 3ms/step - loss: 0.3963 - accuracy: 0.9781
Epoch 4/20
2/2 [==============================] - 0s 2ms/step - loss: 0.3152 - accuracy: 0.9825
Epoch 5/20
2/2 [==============================] - 0s 2ms/step - loss: 0.2407 - accuracy: 0.9912
Epoch 6/20
2/2 [==============================] - 0s 2ms/step - loss: 0.1814 - accuracy: 0.9825
Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1289 - accuracy: 0.9956
Epoch 8/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0892 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0598 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0413 - accuracy: 0.9956
Epoch 11/20
2/2 [==============================] -

2/2 [==============================] - 0s 2ms/step - loss: 0.7138 - accuracy: 0.5088
Epoch 2/20
2/2 [==============================] - 0s 3ms/step - loss: 0.5343 - accuracy: 0.9605
Epoch 3/20
2/2 [==============================] - 0s 4ms/step - loss: 0.4251 - accuracy: 0.9825
Epoch 4/20
2/2 [==============================] - 0s 4ms/step - loss: 0.3355 - accuracy: 0.9868
Epoch 5/20
2/2 [==============================] - 0s 2ms/step - loss: 0.2544 - accuracy: 0.9912
Epoch 6/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1890 - accuracy: 0.9956
Epoch 7/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1334 - accuracy: 0.9956
Epoch 8/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0993 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0697 - accuracy: 0.9912
Epoch 10/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0493 - accuracy: 0.9956
Epoch 11/20
2/2 [==============================] -

2/2 [==============================] - 0s 2ms/step - loss: 0.6262 - accuracy: 0.6711
Epoch 2/20
2/2 [==============================] - 0s 3ms/step - loss: 0.4926 - accuracy: 0.9474
Epoch 3/20
2/2 [==============================] - 0s 3ms/step - loss: 0.3955 - accuracy: 0.9781
Epoch 4/20
2/2 [==============================] - 0s 3ms/step - loss: 0.3005 - accuracy: 0.9825
Epoch 5/20
2/2 [==============================] - 0s 3ms/step - loss: 0.2194 - accuracy: 0.9912
Epoch 6/20
2/2 [==============================] - 0s 3ms/step - loss: 0.1590 - accuracy: 0.9956
Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1113 - accuracy: 0.9956
Epoch 8/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0738 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0478 - accuracy: 0.9956
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0346 - accuracy: 0.9956
Epoch 11/20
2/2 [==============================] -

...

Epoch 6/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0464 - accuracy: 1.0000
Epoch 7/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0341 - accuracy: 1.0000
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0244 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 11/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 13/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 14/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 15/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 16/20
2/2 [=================

2/2 [==============================] - 0s 3ms/step - loss: 0.0461 - accuracy: 0.9956
Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0351 - accuracy: 0.9956
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0264 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 11/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 13/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 14/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 15/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 16/20
2/2 [============================

Epoch 7/20
2/2 [==============================] - 0s 4ms/step - loss: 0.1126 - accuracy: 0.9825
Epoch 8/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0434 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0605 - accuracy: 0.9868
Epoch 10/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0477 - accuracy: 0.9868
Epoch 11/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0313 - accuracy: 0.9912
Epoch 12/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0719 - accuracy: 0.9912
Epoch 13/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0474 - accuracy: 0.9956
Epoch 14/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0257 - accuracy: 0.9956
Epoch 15/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0537 - accuracy: 0.9912
Epoch 16/20
2/2 [==============================] - 0s 5ms/step - loss: 0.0192 - accuracy: 0.9956
Epoch 17/20
2/2 [================

2/2 [==============================] - 0s 2ms/step - loss: 0.0568 - accuracy: 0.9912
Epoch 8/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0477 - accuracy: 0.9956
Epoch 9/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0429 - accuracy: 0.9868
Epoch 10/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0377 - accuracy: 0.9868
Epoch 11/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0546 - accuracy: 0.9912
Epoch 12/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0543 - accuracy: 0.9868
Epoch 13/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0355 - accuracy: 0.9956
Epoch 14/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0310 - accuracy: 0.9912
Epoch 15/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0169 - accuracy: 0.9956
Epoch 16/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0271 - accuracy: 0.9956
Epoch 17/20
2/2 [===========================

2/2 [==============================] - 0s 4ms/step - loss: 0.0328 - accuracy: 1.0000
Epoch 8/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0235 - accuracy: 1.0000
Epoch 9/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0179 - accuracy: 1.0000
Epoch 10/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 11/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0112 - accuracy: 1.0000
Epoch 12/20
2/2 [==============================] - 0s 3ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 13/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 14/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 15/20
2/2 [==============================] - 0s 2ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 16/20
2/2 [==============================] - 0s 4ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 17/20
2/2 [===========================

In [32]:
"""Note that these values might differ a little from the ones in the report due to randomness"""

for opt in optimizers:
    
    accuracies,losses = results[opt]
    
    print("-------------------" + opt + "-------------------")
    print("Mean accuracy: " + str(np.mean(accuracies)))
    print("Mean loss: " + str(np.mean(losses)))
    print("Std loss: " + str(np.std(losses)))
    print(" ")


-------------------adam-------------------
Mean accuracy: 0.9941227942705154
Mean loss: 0.03437008012842853
Std loss: 0.040759623693379904
 
-------------------sgd-------------------
Mean accuracy: 0.9299999988079071
Mean loss: 0.4758161693811417
Std loss: 0.03955930201326413
 
-------------------rmsprop-------------------
Mean accuracy: 0.9947368276119232
Mean loss: 0.024973333940142766
Std loss: 0.019675161194686056
 
